In [11]:
!pip3 install python-docx



Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 239 kB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 8.5 MB 26.9 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [10]:
from googleapiclient.discovery import build
import json

# Replace the following with your API key and playlist ID
api_key = ''
playlist_id = ''

youtube = build('youtube', 'v3', developerKey=api_key)

def get_playlist_videos(playlist_id):
    videos = []
    video_ids_seen = set()  # To keep track of video IDs already seen
    next_page_token = None

    while True:
        pl_request = youtube.playlistItems().list(
            part='snippet',
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        )

        pl_response = pl_request.execute()

        for item in pl_response['items']:
            video_id = item['snippet']['resourceId']['videoId']
            title = item['snippet']['title']
            url = f'https://www.youtube.com/watch?v={video_id}'

            if video_id not in video_ids_seen:
                video_ids_seen.add(video_id)
                videos.append({"title": title, "url": url})
        
        next_page_token = pl_response.get('nextPageToken')

        if not next_page_token:
            break

    return videos

unique_videos = get_playlist_videos(playlist_id)

# Saving the unique videos to a JSON file
with open('unique_videos.json', 'w') as json_file:
    json.dump(unique_videos, json_file, indent=4)

print(f"Exported {len(unique_videos)} unique videos to unique_videos.json.")


Exported 123 unique videos to unique_videos.json.


In [4]:
import json
import os
from docx import Document

# Replace this with the path to your 'ad_summaries' directory
ad_summaries_dir = '/Users/mtbutler/Desktop/Campaign Qualtrics/ad_summaries'

# Function to read the text from a Word document
def read_docx(file_path):
    doc = Document(file_path)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)

# Load the existing JSON data
with open('unique_videos.json', 'r') as json_file:
    videos = json.load(json_file)

# Function to adjust the directory name based on the title
def adjust_dir_name(title):
    # Replace spaces with hyphens and remove 'wm' suffix if present
    adjusted_title = title.replace(" ", "-")
    if adjusted_title.endswith("-wm"):
        adjusted_title = adjusted_title[:-3]  # Remove the last 3 characters ("-wm")
    return adjusted_title

# Iterate through each video in the JSON file
for video in videos:
    title = video["title"]
    # Adjust the directory name based on the title
    dir_name = adjust_dir_name(title)
    full_dir_path = os.path.join(ad_summaries_dir, dir_name)
    
    # Initialize summaries in video object
    video['summary_1'] = ''
    video['summary_2'] = ''
    video['summary_3'] = ''

    # Check if the directory exists
    if os.path.exists(full_dir_path):
        # Iterate through each summary file and update the video object
        for summary_name in ['summary_1.docx', 'summary_2.docx', 'summary_3.docx']:
            summary_path = os.path.join(full_dir_path, summary_name)
            if os.path.isfile(summary_path):
                video[summary_name.split('.')[0]] = read_docx(summary_path)

# Write the updated data back to the JSON file
with open('unique_videos_updated.json', 'w') as json_file:
    json.dump(videos, json_file, indent=4)

print("Updated summaries have been added to unique_videos_updated.json.")




Updated summaries have been added to unique_videos_updated.json.


In [1]:
import json
import random

# Step 1: Read the JSON data from unique_videos_updated.json
with open('unique_videos_updated.json', 'r') as file:
    data = json.load(file)

updated_data = []

for item in data:
    # Filter summaries to include only non-empty ones and pair them with their number (1, 2, or 3)
    summaries = [(key[-1], value) for key, value in item.items() if key.startswith('summary') and value]
    # Randomly select two summaries, duplicate if necessary
    selected_summaries = random.sample(summaries, 2) if len(summaries) > 1 else summaries * 2

    # Construct the new object format
    new_item = {
        "title": item["title"],
        "url": item["url"],
        "A": selected_summaries[0][1],
        "B": selected_summaries[1][1],
        "selection": [int(selected_summaries[0][0]), int(selected_summaries[1][0])]
    }
    updated_data.append(new_item)

# Step 3: (Optional) Write the processed data back to a new JSON file
with open('unique_videos_processed.json', 'w') as outfile:
    json.dump(updated_data, outfile, indent=4)

print("Processing complete. Data saved to 'unique_videos_processed.json'.")


Processing complete. Data saved to 'unique_videos_processed.json'.


In [1]:
import csv
import json

# Path to your JSON file - update this to the location of your JSON file
json_file_path = 'unique_videos_processed.json'

# Path to where you want the CSV file to be saved - update this as needed
csv_file_path = 'videos_for_qualtrics.csv'

# Function to safely encode text for CSV
def encode_for_csv(text):
    if isinstance(text, list):  # Convert list to string if necessary
        return ';'.join(str(num) for num in text)
    return text.replace('"', '""')  # Escape double quotes by doubling them

# Read the JSON data
with open(json_file_path, 'r', encoding='utf-8') as json_file:
    json_data = json.load(json_file)

# Write the JSON data to a CSV file
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file, quoting=csv.QUOTE_ALL)  # Use QUOTE_ALL to handle commas and quotes in text
    # Write the header row
    writer.writerow(['Title', 'URL', 'Summary_A', 'Summary_B', 'Selection'])

    for video in json_data:
        writer.writerow([
            encode_for_csv(video.get('title', '')),
            encode_for_csv(video.get('url', '')),
            encode_for_csv(video.get('A', '')),
            encode_for_csv(video.get('B', '')),
            encode_for_csv(video.get('selection', ''))
        ])
